# EDA using statistical tests

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# stats
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from scipy.stats import levene, shapiro


In [ ]:
import sys

sys.path.append(str(Path("..").resolve()))

import utils


In [ ]:
cleaned_path = Path("../datasets/cleaned.csv")

cars = pd.read_csv(cleaned_path)
cars.head()

## ANOVA (one way)

In [ ]:
interested_feature = 'fuel'
target = 'selling_price'

### Log transformation and drop outliers for the target

Let's take the log of the selling_price

In [ ]:
cars[target].hist()

In [ ]:
cars[target] = np.log(cars[target])
cars.head()

In [ ]:
cars[target].describe()

now we will drop the outliers based on the target variable

In [ ]:
cars[target].hist()

In [ ]:
# remove outliers in the target (selling_price) using the IQR rule
n_before = len(cars)

Q1 = cars[target].quantile(0.25)
Q3 = cars[target].quantile(0.75)
IQR = Q3 - Q1
lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

mask = cars[target].between(lower, upper)
n_out = (~mask).sum()
print(f'Removing {n_out} outlier rows ({n_out/n_before:.2%})')

# overwrite cars with the filtered dataframe
cars = cars[mask].reset_index(drop=True)

In [ ]:
cars[target].hist()

In [ ]:
order = cars[interested_feature].value_counts().index
utils.draw_box_plot(cars, interested_feature, target, order=order)


### Testing fuel feature

we will tests whether the means of a selling_price (the target) differ across two or more groups defined by a fuel feature.

the ANOVA (one way) results will help as determine the best encoding method for this feature.

#### First we will try all four groups

In [ ]:
cars[interested_feature].value_counts()

In [ ]:
order = cars[interested_feature].value_counts().index
utils.draw_box_plot(cars, interested_feature, target, order=order)

In [ ]:
summary_table = cars.groupby(interested_feature)[target].agg(['mean', 'median', 'std', 'count'])
summary_table

In [ ]:
formula = f'{target} ~ C({interested_feature})'
model = smf.ols(formula, data=cars).fit()
anova_results = anova_lm(model)
print(anova_results)

We can see the following:
- the F statistics is large with 248.04 value
- the p-value is very small and below 0.05

##### Check Assumptions

##### 1. Independence of Observations

Since each observation (car) is independent of another, this assumption is met.

##### 2. Homogeneity of variances — Levene’s test

check if the variance in for each group is approximtly equal

In [ ]:
groups = [grp[target].values for _, grp in cars.groupby(interested_feature)]
stat, p_levene = levene(*groups)
print('Levene p-value:', p_levene)

p_levene < 0.05 so variances are unequal (assumption violated).

##### 3. Residual normality — Shapiro (on residuals) + Q–Q plot

check if residuals within each group are normally distributed

In [ ]:
import statsmodels.api as sm

residuals = model.resid

# Shapiro test (small samples only; for large n it's almost always significant)
stat_sh, p_sh = shapiro(residuals)
print('Shapiro p-value:', p_sh)

# Q-Q plot
sm.qqplot(residuals, line='45', fit=True)
plt.title('Q-Q plot of residuals')
plt.show()

from the graph, we can see that the point is almost on the line (light tail normal distribution), so we can accept it with this p-value because n is large so Shapiro test is almost always significant on large datasets

#### Welch ANOVA

We will try Welch ANOVA because the varriance assumption is violated

In [ ]:
import pingouin as pg
welch = pg.welch_anova(dv=target, between=interested_feature, data=cars)
print(welch)

Alright, the p-value < 0.05 => we can reject the null hypothesis => the group have different effect on the target

#### Post Hoc

to find which specific groups differ, 

In [ ]:
tukey = pairwise_tukeyhsd(endog=cars[target], groups=cars[interested_feature], alpha=0.05)
print(tukey.summary())


#### Let's try to merge the CNG and Petrol feature togather

In [ ]:
# merge the 'Pertorl' and 'CNG' categories into a single 'Other' category
df = cars.copy()
df[interested_feature] = df[interested_feature].replace({'Petrol': 'Other', 'CNG': 'Other'})
df[interested_feature].value_counts()

In [ ]:
anova_res = utils.anova_full_report(
    df,
    interested_feature,
    target,
    order_by_frequency=True,
    show_plots=True,
    verbose=True
)

#### Let's try to merge the LPG and CNG groups

In [ ]:
df = cars.copy()
df[interested_feature] = df[interested_feature].replace({'LPG': 'Other', 'CNG': 'Other'})
df[interested_feature].value_counts()

In [ ]:
anova_res = utils.anova_full_report(
    df,
    interested_feature,
    target,
    order_by_frequency=True,
    show_plots=True,
    verbose=True
)

### Testing owner feature

In [ ]:
interested_feature = 'owner'

In [ ]:
cars[interested_feature].value_counts()

In [ ]:
order = cars[interested_feature].value_counts().index
utils.draw_box_plot(cars, interested_feature, target, order=order)

We will drop the row with 'Test Drive Car' value

In [ ]:
# We will drop the row with 'Test Drive Car' value
cars = cars[cars[interested_feature] != 'Test Drive Car'].reset_index(drop=True)
cars[interested_feature].value_counts()

In [ ]:
summary_table = cars.groupby(interested_feature)[target].agg(['mean', 'median', 'std', 'count'])
summary_table

In [ ]:
anova_res = utils.anova_full_report(
    cars,
    interested_feature,
    target
)

Let's try to merge the 'Third Owner' and 'Fouth & Above Owner' to 'other'

In [ ]:
df = cars.copy()

df[interested_feature] = df[interested_feature].replace({
    'Third Owner': 'Third & Above Owner',
    'Fourth & Above Owner': 'Third & Above Owner'
})

df[interested_feature].value_counts()

In [ ]:
summary_table = df.groupby(interested_feature)[target].agg(['mean', 'median', 'std', 'count'])
summary_table

In [ ]:
order = df[interested_feature].value_counts().index
utils.draw_box_plot(df, interested_feature, target, order=order)

In [ ]:
res_anova = utils.anova_full_report(df, interested_feature, target)

### Conclusions

- We will use one-hot encoding to the feature 'fuel' after grouping it: Petrol, Diseil, other
- we will use ordinal encoding to the feature 'owner' after grouping it: First Owner, Second Owner, Third & Above Owner

# Chi-square

check association between two categorical features using χ² (and sensible fallbacks / diagnostics): contingency table, χ² test (with Yates correction when appropriate), Fisher exact for 2×2 small tables, permutation p-value (optional), expected counts check, standardized residuals + heatmap, Cramér’s V (effect size)

In [ ]:
cars.describe(include='object')

In [ ]:
cars['seller_type'].value_counts()

In [ ]:
cars['owner'].value_counts()

In [ ]:
utils.chi2_association_report(cars, 'owner', 'seller_type')

We can see weak association between the two features because Cramer's V < 0.3 